In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [67]:
data = pd.read_csv('D:/farzad/Artificial Intelligence/Sem2/NLP/Consumer_Complaints.csv')



In [68]:
data.dropna(inplace=True)
X = data['Consumer complaint narrative']

y = data['Product']

In [69]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#word = word_tokenize((X.items))


#stops = set(stopwords.words('english')+'xxxx'+'xxx'+'xx')
#x = [w for w in word if w not in stops]

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer


#,'xxxx','xxx','xx'
tfidf = TfidfVectorizer(stop_words='english')
vector = tfidf.fit_transform(X)

In [80]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
out = le.fit_transform(y)

In [81]:
from sklearn.decomposition import TruncatedSVD

In [82]:
svd = TruncatedSVD(n_components=5, algorithm='randomized', n_iter=1000, random_state=4)

In [83]:
svd.fit(vector)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=1000,
       random_state=4, tol=0.0)

In [84]:
len(svd.components_)

5

In [85]:
terms = tfidf.get_feature_names()
topics = []
for i, comp in enumerate(svd.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    topics.append("Topic "+str(i)+": ")
    for t in sorted_terms:
        topics.append(t[0])
final_topic_list = [topics[i:i+8] for i in range(0, len(topics), 8)]
for x in final_topic_list:
    print (x)

['Topic 0: ', 'xxxx', 'xx', 'debt', 'credit', 'account', 'collection', '00']
['Topic 1: ', 'xx', 'debt', 'credit', 'report', 'account', 'company', 'collection']
['Topic 2: ', 'xx', 'xxxx', '00', '2016', 'loans', '2017', 'dated']
['Topic 3: ', 'calls', 'phone', 'number', 'calling', 'day', 'times', 'xx']
['Topic 4: ', '00', 'payment', 'pay', 'loan', 'paid', 'payments', 'told']


In [86]:
import re
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel



# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from collections import Counter
import operator



In [123]:
text = list(X.values)

text = text[:1000]

stop = list(stopwords.words('english'))
exclude = set(punctuation)
lemma = WordNetLemmatizer()
stop=stop+['x','xx','xxx','xxxx','xxxxxxxx','xxxxxxxx']

In [124]:
text = [str(x).lower() for x in text]
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [122]:

#word=word_tokenize(str(x) for x in text)
#word

In [125]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized  = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in text]
print(doc_clean[:2])

[['called', 'cell', 'phone', 'first', 'national', 'debt', 'inc', 'told', 'owed', '21000', 'debt', 'incurred', 'xxxxxxxx', 'xxxx', 'debt', 'never', 'resolved', 'time', 'returned', 'home', 'found', 'debt', 'longer', 'bank', 'moved', 'something', 'website', 'number', 'recording', 'find', 'bought', 'another', 'bank', 'debt', 'came', 'xxxxxxxx', 'xxxxxxxx', 'xxxxxxxx', 'xxxxxxxx', 'debt', 'nine', 'year', 'old'], ['sent', 'cease', 'desist', 'letter', 'medical', 'debt', 'collection', 'agency', 'bill', 'recently', 'deceased', 'husband', 'ca', 'nt', 'afford', 'pay', 'certified', 'mail', 'receipt', 'payment', 'made', 'never', 'got', 'receipt', 'back', 'get', '2', 'robo', 'collection', 'call', 'day', '4', 'day', 'week', 'want', 'call', 'stop']]


In [126]:
import gensim
from gensim import corpora


In [127]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [128]:
print ([[(dictionary[id], freq) for id, freq in cp] for cp in doc_term_matrix[:1]])

[[('21000', 1), ('another', 1), ('bank', 2), ('bought', 1), ('called', 1), ('came', 1), ('cell', 1), ('debt', 6), ('find', 1), ('first', 1), ('found', 1), ('home', 1), ('inc', 1), ('incurred', 1), ('longer', 1), ('moved', 1), ('national', 1), ('never', 1), ('nine', 1), ('number', 1), ('old', 1), ('owed', 1), ('phone', 1), ('recording', 1), ('resolved', 1), ('returned', 1), ('something', 1), ('time', 1), ('told', 1), ('website', 1), ('xxxx', 1), ('xxxxxxxx', 5), ('year', 1)]]


In [129]:
lda = gensim.models.ldamodel.LdaModel
ldamodel = lda(corpus = doc_term_matrix, num_topics = 5, id2word = dictionary, passes = 50)
print(ldamodel.print_topics(num_topics=5,num_words = 7))

[(0, '0.024*"loan" + 0.021*"xxxx" + 0.010*"student" + 0.009*"well" + 0.009*"xxxxxxxx" + 0.008*"u" + 0.007*"payment"'), (1, '0.031*"call" + 0.017*"xxxx" + 0.016*"debt" + 0.016*"number" + 0.014*"phone" + 0.012*"called" + 0.011*"told"'), (2, '0.020*"xxxx" + 0.016*"credit" + 0.016*"payment" + 0.014*"collection" + 0.013*"bill" + 0.012*"account" + 0.011*"paid"'), (3, '0.037*"debt" + 0.029*"credit" + 0.026*"xxxx" + 0.020*"account" + 0.017*"report" + 0.016*"collection" + 0.011*"company"'), (4, '0.021*"xxxx" + 0.007*"s" + 0.007*"number" + 0.006*"bankruptcy" + 0.005*"debt" + 0.004*"court" + 0.004*"service"')]


In [130]:
!pip install pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\Farzad Avari\AppData\Local\conda\conda\envs\ImageEnv1\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      37.269428        1       1  0.075416  0.035929
3      27.796286        1       2  0.124283 -0.108214
1      21.266537        1       3  0.022214  0.144463
0       8.568417        1       4 -0.117004  0.003453
4       5.099335        1       5 -0.104909 -0.075632, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
354   Default   615.000000         call   615.000000  30.0000  30.0000
4167  Default   385.000000         loan   385.000000  29.0000  29.0000
2269  Default  1557.000000         xxxx  1557.000000  28.0000  28.0000
5805  Default   470.000000       report   470.000000  27.0000  27.0000
5994  Default  1106.000000       credit  1106.000000  26.0000  26.0000
1616  Default  1323.000000         debt  1323.000000  25.0000  25.0000
3121  Default   411.000000       number   411.000000  24.0000  24.0000
5995  Default   178.000000      calling   178.000000  23.0000  23.0000
2255  Default   361.000000        phone   361.000000  22.0000  22.0000
242   Default   566.000000      payment   566.000000  21.0000  21.0000
4636  Default   423.000000         bill   423.000000  20.0000  20.0000
1810  Default   398.000000       called   398.000000  19.0000  19.0000
1322  Default   136.000000         well   136.000000  18.0000  18.0000
4274  Default    89.000000      student    89.000000  17.0000  17.0000
554   Default   338.000000     xxxxxxxx   338.000000  16.0000  16.0000
2519  Default   377.000000         told   377.000000  15.0000  15.0000
38    Default   831.000000      account   831.000000  14.0000  14.0000
5117  Default   275.000000            s   275.000000  13.0000  13.0000
4641  Default   139.000000        court   139.000000  12.0000  12.0000
2140  Default   139.000000            u   139.000000  11.0000  11.0000
1583  Default   773.000000   collection   773.000000  10.0000  10.0000
6078  Default   152.000000    reporting   152.000000   9.0000   9.0000
292   Default   340.000000      service   340.000000   8.0000   8.0000
2782  Default   356.000000  information   356.000000   7.0000   7.0000
1299  Default    95.000000     attorney    95.000000   6.0000   6.0000
2293  Default   277.000000         said   277.000000   5.0000   5.0000
5160  Default   374.000000       agency   374.000000   4.0000   4.0000
255   Default    44.000000        fargo    44.000000   3.0000   3.0000
3088  Default   492.000000        would   492.000000   2.0000   2.0000
3533  Default   206.000000         bank   206.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
5117   Topic5    26.525452            s   275.036804   0.6373  -4.9671
5058   Topic5    10.123495   discharged    40.597870   1.5872  -5.9303
2922   Topic5    15.773570          see   100.635292   1.1229  -5.4868
5700   Topic5     6.958164         free    19.054798   1.9687  -6.3052
1299   Topic5    15.108974     attorney    95.750206   1.1296  -5.5299
167    Topic5    12.032640         note    60.354252   1.3635  -5.7575
5459   Topic5    11.885350     personal    61.744522   1.3284  -5.7699
4641   Topic5    16.469097        court   139.054520   0.8427  -5.4437
3121   Topic5    25.021845       number   411.425537   0.1762  -5.0254
1818   Topic5     8.984564      statute    36.196747   1.5826  -6.0497
649    Topic5    12.864676       trying   135.908997   0.6186  -5.6907
292    Topic5    16.366730      service   340.566650  -0.0593  -5.4499
13     Topic5     7.489955     daughter    25.349297   1.7569  -6.2316
2782   Topic5    15.870643  information   356.996368  -0.1372  -5.4807
3480   Topic5    11.232305          new   108.738022   0.7059  -5.8264
3709   Topic5    15.248029        never   377.656067  -0.2335  -5.5207
1616   Topic5    19.753624         debt  1323.332031  -1.2285  -5.2618
4423   Topic5    12.288153   